In [ ]:
!pip install torch torchvision
!pip install Pillow==4.1.1
!pip install image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 94.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.2/112.2 KB 15.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for Pillow: filename=Pillow-4.1.1-cp39-cp39-linux_x86_64.whl size=1079484 sha256=de52e71c81a656fd7bb56233c83b52c728c6d45c7da0ea1a6f57b8a21e6446c9
  Stored in directory: /root/.cache/pip/wheels/2c/eb/0e/13ca6f7d7c9b914ee449007ce3bf93c01b5d5433d7ac23606a
  Created wheel for olefile: filename=olefile-0.46-py2.py3-none-any.whl size=35432 sha256=ca474b3b2b06074750f14d091cde867cd07ea6801b0757f00df32a3608b8f795
  Stored in directory: /root/.cache/pip/wheels/64/b8/ba/ebba30390fbd997074f35e42a842ce3fd933213cac8753414e
Successfully built Pillow o

In [ ]:
!pip install torchvision

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

import torchvision.transforms as transforms
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.utils as utils

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
def load_image(image_path, transform=None, max_size=None, shape=None):
    image = Image.open(image_path)
    
    if max_size is not None:
        # Resize the image to the specified maximum size
        width, height = image.size
        if width > height:
            new_width = max_size
            new_height = int(max_size * height / width)
        else:
            new_height = max_size
            new_width = int(max_size * width / height)
        image = image.resize((new_width, new_height), Image.ANTIALIAS)
    
    if shape is not None:
        # Resize the image to the specified shape
        image = image.resize(shape, Image.LANCZOS)
    
    if transform is not None:
        # Apply the specified transform to the image
        image = transform(image).unsqueeze(0)
    
    return image.to(device)

In [ ]:
def gram_matrix(input_tensor):
    batch_size, channels, height, width = input_tensor.size()
    features = input_tensor.view(batch_size * channels, height * width)
    gram = torch.mm(features, features.t())
    return gram.div(batch_size * channels * height * width)

In [ ]:
content_img = Image.open('path_for_input_image_file.jpg')
style_img = Image.open('path_for_album_image_file.jpg')

In [ ]:
content_path = 'path_for_input_image_file.jpg'
style_path = 'path_for_album_image_file.jpg'

In [ ]:
transform = transforms.Compose([
    transforms.Resize(512),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])

content_img = transform(content_img)
style_img = transform(style_img)

In [ ]:
print(content_img.shape)

torch.Size([3, 512, 528])


In [ ]:
input_img = content_img.clone()
batch_size = 1
content_img = content_img.repeat(batch_size, 1, 1).to(device)
style_img = style_img.repeat(batch_size, 1, 1).to(device)
input_img = input_img.repeat(batch_size, 1, 1).to(device)

In [ ]:
cnn = models.vgg19(pretrained=True).features.to(device).eval()

/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/vgg19-dcbb9e9d.pth" to /root/.cache/torch/hub/checkpoints/vgg19-dcbb9e9d.pth


  0%|          | 0.00/548M [00:00<?, ?B/s]

In [ ]:
def get_features(image, model, layers=None):
    if layers is None:
        layers = {'0': 'conv1_1',
                  '5': 'conv2_1',
                  '10': 'conv3_1',
                  '19': 'conv4_1',
                  '21': 'conv4_2',  # content layer
                  '28': 'conv5_1'}
    features = {}
    x = image.unsqueeze(0)
    for name, layer in model._modules.items():
        x = layer(x)
        if name in layers:
            features[layers[name]] = x
    return features

In [ ]:
content_img = content_img.to(device)
style_img = style_img.to(device)

In [ ]:
content_features = get_features(content_img.squeeze(0), cnn)
style_features = get_features(style_img.squeeze(0), cnn)

In [ ]:
def gram_matrix(tensor):
    _, c, h, w = tensor.size()
    tensor = tensor

In [ ]:
def gram_matrix(tensor):
    _, c, h, w = tensor.size()
    tensor = tensor.view(c, h * w)
    gram = torch.mm(tensor, tensor.t())
    return gram

In [ ]:
content_weight = 10  # alpha
style_weight = 100  # beta

In [ ]:
input_img = content_img.clone().requires_grad_(True)

In [ ]:
optimizer = torch.optim.Adam([input_img], lr=0.01)
mse_loss = torch.nn.MSELoss()

In [ ]:
num_steps = 2000
style_layers = ['conv1_1', 'conv2_1', 'conv3_1', 'conv4_1', 'conv5_1']

# Run the style transfer algorithm
for i in range(num_steps):
    input_features = get_features(input_img, cnn)
    
    content_loss = mse_loss(input_features['conv4_2'], content_features['conv4_2'])
    
    style_loss = 0
    for layer in style_layers:
        input_feature = input_features[layer]
        _, c, h, w = input_feature.size()
        input_gram = gram_matrix(input_feature)
        style_gram = gram_matrix(style_features[layer])
        layer_loss = mse_loss(input_gram, style_gram)
        style_loss += layer_loss / (c * h * w)
    style_loss *= style_weight
    
    total_loss = content_weight * content_loss + style_loss
    
    optimizer.zero_grad()
    total_loss.backward(retain_graph=True)
    optimizer.step()
    
    if i % 100 == 0:
        print(f'Step {i}: total loss = {total_loss.item()}')
        output_img = input_img.clone().detach().cpu()
        output_img = output_img.squeeze(0)
        utils.save_image(output_img, f'path_for_save_output_image_file/output_{i}.jpg')

Step 0: total loss = 13181.6484375
Step 100: total loss = 1833.1669921875
Step 200: total loss = 831.111572265625
Step 300: total loss = 499.0368957519531
Step 400: total loss = 354.6078796386719
Step 500: total loss = 279.9184265136719
Step 600: total loss = 235.51864624023438
Step 700: total loss = 206.00778198242188
Step 800: total loss = 185.20301818847656
Step 900: total loss = 169.9143524169922
Step 1000: total loss = 158.39682006835938
Step 1100: total loss = 149.52151489257812
Step 1200: total loss = 397.13720703125
Step 1300: total loss = 141.52151489257812
Step 1400: total loss = 128.7421875
Step 1500: total loss = 123.49275970458984
Step 1600: total loss = 119.45243835449219
Step 1700: total loss = 115.99407196044922
Step 1800: total loss = 113.03407287597656
Step 1900: total loss = 110.35930633544922
